# 1. 安装依赖包，导入头文件

本样例复现论文：*Practical Deep Reinforcement Learning Approach for Stock Trading （https://arxiv.org/abs/1811.07522）*.

代码参考：[FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials))

原理介绍：

强化学习核心部分包括“机器人”和“环境”。流程大致如下：
- 机器人和环境进行交互，观察到当前的条件，称为“状态”（**state**），并且可以执行“动作”（**action**）
- 机器人执行动作后，会进入一个新的状态，同时，环境给机器人一个反馈，叫奖励（**reward**）
  (通过数字反馈新状态的好坏)
- 之后，机器人和环境不停的重复交互，机器人要尽可能多的获取累计奖励

强化学习是一种方法，让机器人学会提升表现，并达成目标。

实现介绍

使用OpenAI gym的格式构建股票交易的环境。

state-action-reward的含义如下：

- **State s**: 状态空间表示机器人对环境的感知。就像人工交易员分析各种信息和数据。机器人从历史数据
  观察交易价格以及技术指标。通过和环境交互进行学习（一般通过回放历史数据）
  
- **Action a**: 动作空间代码机器人在每个状态可以执行的动作。例如，a ∈ {−1, 0, 1}, −1, 0, 1代表
  卖出、持仓、买入。当处理多支股票时，a ∈{−k, ..., −1, 0, 1, ..., k}, 比如，“买10股AAPL”或者
  “卖出10股AAPL”即10或-10。

- **Reward function r(s, a, s′)**: 奖励用于激励机器人学习一个更好的策略。例如，在状态s下执行动作a
  以改变投资组合值，并到达一个新的状态s', 例如，r(s, a, s′) = v′ − v, v′ 和 v 代表状态分别在s′ 
  和s时的投资组合总市值。
  
- **Market environment**: 道琼斯工业平均指数（DJIA）中30只成分股，包含回测时间段的所有交易数据。

In [1]:
# 如果没有安装，解注释进行安装
# !pip install -r requirements.txt

## 请把下面解注释，安装finrl库
# 或者把如下Github仓中的finrl文件夹考到根目录即可使用
##!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

# 强化学习库，使用stable_baselines3
# 注意：
#    1. 强化学习比起机器学习慢很多，CPU训练大约2分钟，强化学习使用单卡GPU大约需要30分钟左右完成训练
#    2. 强化学习不稳定，每次收敛的loss不一样，且效果可能差异大     

In [2]:
import pandas as pd

from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

In [3]:
from finrl import config  # 包含各类超参
from finrl import config_tickers  # 常见各类市场股票代码集合，比如中证300
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
# 创建目录
check_and_make_directories([TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

# 2. 在OpenAI Gym-style构建市场环境

In [4]:
# 读取训练数据，所有股票均混在了一个csv表里，格式如下
# 索引     日期          股票
#  0      2009-01-02    苹果
#  0      2009-01-02    亚马逊
#  1      2009-01-05    苹果
#  1      2009-01-05    亚马逊

# 注意：必须保持上述该格式，同样的索引下至少有2个数据，否则会报错，
# 原因：
#   1. 在finrl/meta/env_stock_trading/env_stocktrading.py的
#      _initiate_state函数中self.data.close.values.tolist()，
#      在404行，要求self.data.close必须是二维数组
#   2. 而finrl/meta/env_stock_trading/env_stocktrading.py的
#      __init__的64行self.data = self.df.loc[self.day, :]，
#      如果索引顺序排，0，1，2。。。，会导致只取到一个行数，一维
#      数据传入导致第1点中所述的问题
#      （因此，如果只有一支股票时，需要把索引全部改成一样的，当然
#      这种情况几乎不存在，也可以暂时忽略）
# 解决方法：
# 1. 降低numpy版本
# 2. 把数据改成二维的，即（10，）-》（1，10） （改完是否存在回测不完整性，没有详细验证）
# 3. 保持最上方所示的数据格式（推荐）

train = pd.read_csv(os.path.join(DATA_SAVE_DIR, 'train_data.csv'))
train = train.set_index(train.columns[0]) # 第一列为索引
train.index.names = ['']
assert train.shape[0] > 1, '数据必须至少包含2行，即2天以上'

In [5]:
# 默认定义了8个技术因子
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [6]:
# 共29支股票，状态空间291
stock_dimension = len(train.tic.unique())
# 状态说明
# 1：账户余额
# [1: stock_dimension+1]: 股票价格
# [stock_dimension+1: 1 + 2*stock_dimension]: 持仓数量
# len(INDICATORS)*stock_dimension：每支股票的因子状态，bool型表示
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [7]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension  # 手续费
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## 2.1 构建训练环境

In [8]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# 3. 训练深度强化学习模型
* 强化学习库：使用 **Stable Baselines 3**. 也可以尝试更换 **ElegantRL** and **Ray RLlib**.
* FinRL库包含精调的标准深度强化学习算法, 包括DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3.

In [9]:
# 选择需要使用的强化学习算法

# 5种算法：A2C, DDPG, PPO, TD3, SAC
if_using_a2c = True
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = True

## Agent 1: A2C


In [10]:
if if_using_a2c:
    agent = DRLAgent(env = env_train)
    model_a2c = agent.get_model("a2c")

    # set up logger
    tmp_path = RESULTS_DIR + '/a2c'
    new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set new logger
    model_a2c.set_logger(new_logger_a2c)
    
    trained_a2c = agent.train_model(model=model_a2c, 
                                 tb_log_name='a2c',
                                 total_timesteps=50000)

    save_path = os.path.join(TRAINED_MODEL_DIR, 'a2c') 
    trained_a2c.save(os.path.join(save_path, "agent_a2c.zip"))

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
Logging to results/a2c
--------------------------------------
| time/                 |            |
|    fps                | 56         |
|    iterations         | 100        |
|    time_elapsed       | 8          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -41.2      |
|    explained_variance | -0.748     |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -38.2      |
|    reward             | -0.2910683 |
|    std                | 1          |
|    value_loss         | 2.75       |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 69         |
|    iterations         | 200        |
|    time_elapsed       | 14         |
|    total_timesteps    | 1000       |
| train/                |            |
|   

## Agent 2: DDPG

In [11]:
 
if if_using_ddpg:
    agent = DRLAgent(env = env_train)
    model_ddpg = agent.get_model("ddpg")

    # set up logger
    tmp_path = RESULTS_DIR + '/ddpg'
    new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set new logger
    model_ddpg.set_logger(new_logger_ddpg)
    
    trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) 
    
    save_path = os.path.join(TRAINED_MODEL_DIR, 'ddpg') 
    trained_ddpg.save(os.path.join(save_path, "agent_ddpg.zip"))

### Agent 3: PPO

In [12]:
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}

if if_using_ppo:
    agent = DRLAgent(env = env_train)
    model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

    # set up logger
    tmp_path = RESULTS_DIR + '/ppo'
    new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set new logger
    model_ppo.set_logger(new_logger_ppo)
    
    trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=200000)
    
    save_path = os.path.join(TRAINED_MODEL_DIR, 'ppo') 
    trained_ppo.save(os.path.join(save_path, "agent_ppo.zip"))

### Agent 4: TD3

In [13]:
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}


if if_using_td3:
    agent = DRLAgent(env = env_train)
    model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

    # set up logger
    tmp_path = RESULTS_DIR + '/td3'
    new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set new logger
    model_td3.set_logger(new_logger_td3)
    
    trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)
    
    save_path = os.path.join(TRAINED_MODEL_DIR, 'td3') 
    trained_td3.save(os.path.join(save_path, "agent_td3")) 

### Agent 5: SAC

In [14]:
# 在单卡GPU下, batch调大显存占用和利用率也上不去，不确定原因
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

if if_using_sac:
    agent = DRLAgent(env = env_train)
    model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

    # set up logger
    tmp_path = os.path.join(RESULTS_DIR, 'sac') 
    new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set new logger
    model_sac.set_logger(new_logger_sac)
    
    trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=70000)

    save_path = os.path.join(TRAINED_MODEL_DIR, 'sac') 
    trained_sac.save(os.path.join(save_path, 'agent_sac.zip'))

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device
Logging to results\sac
day: 2892, episode: 30
begin_total_asset: 1000000.00
end_total_asset: 4068845.14
total_reward: 3068845.14
total_cost: 21184.91
total_trades: 43718
Sharpe: 0.759
-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 33         |
|    time_elapsed    | 348        |
|    total_timesteps | 11572      |
| train/             |            |
|    actor_loss      | 368        |
|    critic_loss     | 151        |
|    ent_coef        | 0.0967     |
|    ent_coef_loss   | -107       |
|    learning_rate   | 0.0001     |
|    n_updates       | 11471      |
|    reward          | -3.0241768 |
-----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 32        |
| 